In [3]:
import pymortar
import glob
URL = "http://beta-api.mortardata.org"
c = pymortar.Client(URL)

# 1.1.1 For Type relaxation rules

## 1.1.1.1 ApplyRule_UpperClass() 

			Input: A triple pattern t (?x, type, C[n])
			Output: List of superclasses of C[n] (first layer)
            ? A relaxed triple t’ such that if C[n] is a subclass of C[n+1], 
			t’= (?x, type, C[n+1]) ?
            
## 1.1.1.2 ApplyRule_LowerClass() 

			Input: A triple pattern t (?x, type, C[n])
            Output: List of subclasses of C[n] (first layer)
            ?A relaxed triple t’ such that if C[n] is a superclass of C[n+1], 
			t’= (?x, type, C1)?
            
## 1.1.1.3 ApplyRule_SiblingClass()  
            Input: A triple pattern t (?x, type, C[n])
            Output: List of siblingclasses (contain same super class of C[n]) (first layer)

In [443]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, OWL, TAG, RDF
from rdflib import URIRef

brick_graph = brickschema.Graph(load_brick=True)

#From Anand's Code
def get_namespace(x):
    if ':' in x:
        ns = x.split(':')[0]
    else:
        ns = 'brick'
    if ns.lower() == 'brick':
        return BRICK
    elif ns.lower() == 'owl':
        return OWL
    elif ns.lower() == 'tag':
        return TAG
    elif ns.lower() == 'rdf':
        return RDF
    elif ns.lower() == 'rdfs':
        return RDFS
    elif ns.lower() == 'sh':
        return SH
##

def ApplyRule_LowerClass(entity):
    if entity is None:
        return []
    else:
        lower_class = []
        for subj, obj in brick_graph.subject_objects(predicate=RDFS.subClassOf):
            if obj in entity:
                thing = subj.rsplit('#')[-1]
                lower_class.append(thing)
        return lower_class

def ApplyRule_UpperClass(entity):
    upper_class = []
    if entity is None:
        return []
    else:
        super_classes = []
        for sup in brick_graph.transitive_objects(subject=entity, property=RDFS['subClassOf']):
            if isinstance(sup, URIRef):
                thing = sup.rsplit('#')[-1]
                super_classes.append(thing)
        #Retriving first layer of upper_class
        for upper in super_classes:
            sub = BRICK[upper]
            lower_list = ApplyRule_LowerClass(sub)
            entity = entity.rsplit('#')[-1]
            if entity in lower_list:
                upper_class.append(upper)
                
        return upper_class 

def ApplyRule_SiblingClass(entity):
    sibling_class = []
    if entity is None:
        return []
    else:
        upper_class = ApplyRule_UpperClass(entity)
        for upper in upper_class:
            upper = BRICK[upper]
            lower_list = ApplyRule_LowerClass(upper)
            sibling_class.extend(lower_list)
    sibling_class = list(set(sibling_class))
    entity = entity.rsplit('#')[-1]
    sibling_class.remove(entity)
    
    return sibling_class


In [450]:
def test(triple):
    t0 = triple[2]
    #Anands Codes
    if t0[0] != '?':
        sub_ns = get_namespace(t0.split(':')[0])
        sub_entity = t0.split(':')[1]
        sub = sub_ns[sub_entity]

    upper_class = ApplyRule_UpperClass(entity=sub)
    lower_class = ApplyRule_LowerClass(entity=sub)
    sibling_class = ApplyRule_SiblingClass(entity=sub)
    
    return upper_class, lower_class, sibling_class

In [451]:
triple = ["?Sensor",  "rdf:type", "brick:Hot_Water_System_Enable_Command"]
upper, lower, sibling = test(triple)
print("UPPER CLASS:")
print(upper)
print("LOWER CLASS:")
print(lower)
print("SIBLING CLASS:")
print(sibling)

UPPER CLASS:
['System_Enable_Command']
LOWER CLASS:
['Domestic_Hot_Water_System_Enable_Command', 'Discharge_Hot_Water', 'Preheat_Hot_Water_System', 'Heat_Recovery_Hot_Water_System', 'Supply_Hot_Water', 'Reheat_Hot_Water_System', 'Return_Hot_Water', 'Radiation_Hot_Water_System']
SIBLING CLASS:
['Shading_System', 'Chilled_Water_System_Enable_Command', 'Gas_System', 'Electrical_System', 'Lighting_System', 'Safety_System', 'Domestic_Hot_Water_System', 'Heating_Ventilation_Air_Conditioning_System']


In [452]:
triple = ["?Point",  "rdf:type", "brick:Demand_Sensor"]
upper, lower, sibling = test(triple)
print("UPPER CLASS:")
print(upper)
print("LOWER CLASS:")
print(lower)
print("SIBLING CLASS:")
print(sibling)

UPPER CLASS:
['Sensor']
LOWER CLASS:
['Heating_Demand_Sensor', 'Cooling_Demand_Sensor', 'Peak_Power_Demand_Sensor']
SIBLING CLASS:
['Power_Sensor', 'Torque_Sensor', 'Illuminance_Sensor', 'Fire_Sensor', 'Contact_Sensor', 'Energy_Sensor', 'Enthalpy_Sensor', 'Motion_Sensor', 'Conductivity_Sensor', 'Air_Grains_Sensor', 'Current_Sensor', 'Frequency_Sensor', 'Flow_Sensor', 'Adjust_Sensor', 'Dewpoint_Sensor', 'Position_Sensor', 'Rain_Sensor', 'Temperature_Sensor', 'Speed_Sensor', 'Direction_Sensor', 'Duration_Sensor', 'Pressure_Sensor', 'Occupancy_Sensor', 'Heat_Sensor', 'Piezoelectric_Sensor', 'Frost_Sensor', 'Solar_Radiance_Sensor', 'Humidity_Sensor', 'Angle_Sensor', 'Particulate_Matter_Sensor', 'Gas_Sensor', 'Voltage_Sensor', 'Usage_Sensor', 'Hail_Sensor', 'Water_Level_Sensor', 'Luminance_Sensor', 'Capacity_Sensor']


In [453]:
triple = ["?Status",  "rdf:type", 'brick:Differential_Pressure_Load_Shed_Status']
upper, lower, sibling = test(triple)
print("UPPER CLASS:")
print(upper)
print("LOWER CLASS:")
print(lower)
print("SIBLING CLASS:")
print(sibling)

UPPER CLASS:
['Load_Shed_Status', 'Pressure_Status']
LOWER CLASS:
['Chilled_Water_Differential_Pressure_Load_Shed_Status', 'Hot_Water_Differential_Pressure_Load_Shed_Status', 'Medium_Temperature_Hot_Water_Differential_Pressure_Load_Shed_Status']
SIBLING CLASS:
['Hot_Water_Discharge_Temperature_Load_Shed_Status', 'Hot_Water_Supply_Temperature_Load_Shed_Status', 'Supply_Air_Duct_Pressure_Status', 'Discharge_Air_Duct_Pressure_Status']


In [454]:
triple = ["?Equipment",  "rdf:type", 'brick:VAV']
upper, lower, sibling = test(triple)
print("UPPER CLASS:")
print(upper)
print("LOWER CLASS:")
print(lower)
print("SIBLING CLASS:")
print(sibling)

UPPER CLASS:
['Terminal_Unit']
LOWER CLASS:
[]
SIBLING CLASS:
['FCU', 'Constant_Air_Volume_Box', 'Variable_Air_Volume_Box', 'Fan_Coil_Unit', 'Radiator', 'CAV']
